# ノードの停止

---

OpenHPC環境のノードを一時的に停止します。

> このNotebookは AWS, AzureのVCノードに対してのみ利用可能です。他のクラウドプロバイダで実行中のVCノードはノード停止の機能をVCPがサポートしていないためエラーとなります。

## パラメータの指定

ノードの停止を行うのに必要となるパラメータを入力します。

### VCCアクセストークンの入力

VCノードを停止するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### グループ名

OpenHPCのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name =

UnitGroupの変数をgroup_varsファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

## ノードの停止

現在のノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

### 計算ノードの停止

全ての計算ノードを停止します。

In [ ]:
unit_compute = ug.get_unit('compute')
unit_compute.power_off_nodes(num_power_off_nodes=gvars['compute_nodes'])

計算ノードの状態を確認します。全ノードの `node_state` の表示が `POWER_OFF`となっていることを確認してください。

In [ ]:
unit_compute.df_nodes()

### マスターノードの停止

マスターノードを停止します。

> 計算ノードのみを停止したい場合はこの節を実行しないでください。

In [ ]:
unit_master = ug.get_unit('master')
unit_master.power_off_nodes()

マスターノードの状態を確認します。`node_state` の表示が `POWER_OFF`となっていることを確認してください。

In [ ]:
unit_master.df_nodes()